In [1]:
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from einops import rearrange
from torch import  nn

In [ ]:
dino_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14').to('cpu')

# Load image (you can use any standard format like .jpg, .png — ".img" isn't typical)
img = Image.open('/Users/talhaahmed/Library/CloudStorage/OneDrive-HigherEducationCommission/Integration/GitHub/Diffusion-Codes/GMS/Dataset/busi/images/benign_1.png').convert('RGB')  # Ensure it's RGB

# Preprocess to tensor and resize to match model input (e.g., 224x224)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Converts to (C, H, W) and scales to [0,1]
])

# img_tensor = transform(img).unsqueeze(0)  # Add batch dimension -> (1, 3, 224, 224)
img_tensor = transform(img)
img_tensor = (img_tensor.unsqueeze(0)).to('cpu')

result = dino_model(img_tensor, is_training = True)

print(result.keys())

global_cls_feature, local_patch_feature = result['x_norm_clstoken'], result['x_norm_patchtokens'].reshape(1, 16, 16, 384)

print(global_cls_feature.shape, local_patch_feature.shape)

sum(p.numel() for p in dino_model.parameters())

Using cache found in /Users/talhaahmed/.cache/torch/hub/facebookresearch_dinov2_main


dict_keys(['x_norm_clstoken', 'x_norm_regtokens', 'x_norm_patchtokens', 'x_prenorm', 'masks'])
torch.Size([1, 384]) torch.Size([1, 16, 16, 384])


22056576

In [4]:
from networks.sft_pdm import SFTPDM_UNet_DS
from networks.guidance import *
import torch

# Input simulation
images = torch.randn(2, 3, 224, 224)
latent = torch.randn(2, 4, 28, 28)

for guidance_type in ['edge', 'wavelet', 'dino']:
    guidance = prepare_guidance(images, mode=guidance_type)
    if guidance_type == 'wavelet':
        skff_module = SKFF()
        guidance = skff_module(guidance)
    # guidance = torch.nn.functional.interpolate(guidance, size=(28, 28), mode='bilinear', align_corners=False)
    model = SFTPDM_UNet_DS(in_channels=4, out_channels=4, guidance_channels = guidance.shape[1])


    with torch.no_grad():
        out = model(latent, guidance)

    print(f"\n--- {guidance_type.upper()} ---")
    for k, v in out.items():
        print(f"{k}: {v.shape}")



--- EDGE ---
level3: torch.Size([2, 4, 28, 28])
level2: torch.Size([2, 4, 28, 28])
level1: torch.Size([2, 4, 28, 28])
out: torch.Size([2, 4, 28, 28])

--- WAVELET ---
level3: torch.Size([2, 4, 28, 28])
level2: torch.Size([2, 4, 28, 28])
level1: torch.Size([2, 4, 28, 28])
out: torch.Size([2, 4, 28, 28])


Using cache found in /Users/talhaahmed/.cache/torch/hub/facebookresearch_dinov2_main



--- DINO ---
level3: torch.Size([2, 4, 28, 28])
level2: torch.Size([2, 4, 28, 28])
level1: torch.Size([2, 4, 28, 28])
out: torch.Size([2, 4, 28, 28])


In [7]:
from torchinfo import summary
model = SFTPDM_UNet_DS(in_channels=4, out_channels=4, guidance_channels = 3)
print(summary(model, input_size=[(4, 4, 28, 28), (4, 3, 28, 28)]))

Layer (type:depth-idx)                   Output Shape              Param #
SFTPDM_UNet_DS                           [4, 4, 28, 28]            --
├─Conv2d: 1-1                            [4, 32, 28, 28]           1,184
├─ResAttBlock: 1-2                       [4, 32, 28, 28]           --
│    └─ResBlock: 2-1                     [4, 32, 28, 28]           --
│    │    └─Sequential: 3-1              [4, 32, 28, 28]           9,313
│    │    └─Sequential: 3-2              [4, 32, 28, 28]           9,313
│    │    └─Identity: 3-3                [4, 32, 28, 28]           --
│    └─SFT: 2-2                          [4, 32, 28, 28]           --
│    │    └─Sequential: 3-4              [4, 128, 28, 28]          3,584
│    │    └─Conv2d: 3-5                  [4, 32, 28, 28]           36,896
│    │    └─Conv2d: 3-6                  [4, 32, 28, 28]           36,896
│    └─SpatialSelfAttention: 2-3         [4, 32, 28, 28]           --
│    │    └─GroupNorm: 3-7               [4, 32, 28, 28]         

In [ ]:
import React, { useEffect, useMemo, useRef, useState } from "react";
import { Chess } from "chess.js";
import { Button } from "@/components/ui/button";
import { Card, CardContent, CardHeader, CardTitle } from "@/components/ui/card";
import { Dialog, DialogContent, DialogHeader, DialogTitle } from "@/components/ui/dialog";
import { motion, LayoutGroup } from "framer-motion";

/* ------------------------- Assets ------------------------- */
const SPRITE = (id) => `https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/${id}.png`;

const POKEMON_SPRITES = {
  w: { k: SPRITE(493), q: SPRITE(150), r: SPRITE(95), b: SPRITE(282), n: SPRITE(78), p: SPRITE(25) },
  b: { k: SPRITE(491), q: SPRITE(249), r: SPRITE(208), b: SPRITE(197), n: SPRITE(77), p: SPRITE(52) },
};

const CLASSIC_GLYPHS = {
  w: { k: "♔", q: "♕", r: "♖", b: "♗", n: "♘", p: "♙" },
  b: { k: "♚", q: "♛", r: "♜", b: "♝", n: "♞", p: "♟" },
};

/* Optional: swap these with your own hosted SFX (keep CORS in mind). Leave blank "" to use synth fallback. */
const CLASSIC_SFX_URLS = {
  move: "",     // e.g. "https://your.cdn/wood-move.mp3"
  capture: "",  // e.g. "https://your.cdn/wood-capture.mp3"
  check: "",    // e.g. "https://your.cdn/wood-check.mp3"
  win: "",      // e.g. "https://your.cdn/win-chime.mp3"
  draw: "",     // e.g. "https://your.cdn/draw-tone.mp3"
  lose: "",     // e.g. "https://your.cdn/lose-thud.mp3"
};

/* ------------------------- Board helpers ------------------------- */
function useBoard(game) {
  return useMemo(() => {
    const board = game.board();
    const squares = [];
    for (let r = 0; r < 8; r++) {
      for (let c = 0; c < 8; c++) {
        const file = String.fromCharCode(97 + c);
        const rank = 8 - r;
        const square = `${file}${rank}`;
        squares.push({ square, piece: board[r][c], row: r, col: c });
      }
    }
    return squares;
  }, [game.fen()]);
}

/* ------------------------- Theme-aware SFX ------------------------- */
function useChessSfx(theme /* "pokemon" | "classic" */, urls = CLASSIC_SFX_URLS) {
  const ctxRef = useRef(null);
  const buffersRef = useRef({}); // name -> AudioBuffer | null (failed) | undefined (not loaded)

  const ensureCtx = () => {
    if (!ctxRef.current) {
      const Ctx = window.AudioContext || window.webkitAudioContext;
      ctxRef.current = new Ctx();
    }
    return ctxRef.current;
  };

  const loadBuffer = async (name, url) => {
    if (!url) {
      buffersRef.current[name] = null; // mark as unavailable (use fallback)
      return null;
    }
    try {
      const ctx = ensureCtx();
      const res = await fetch(url, { mode: "cors" });
      const arr = await res.arrayBuffer();
      const buf = await ctx.decodeAudioData(arr);
      buffersRef.current[name] = buf;
      return buf;
    } catch {
      buffersRef.current[name] = null; // failed; fallback later
      return null;
    }
  };

  // Preload classic SFX on first interaction
  useEffect(() => {
    const preloader = () => {
      if (theme !== "classic") return;
      Object.entries(urls).forEach(([name, url]) => {
        if (buffersRef.current[name] === undefined) {
          loadBuffer(name, url);
        }
      });
    };
    // start decoding after user gesture (unlocks audio on iOS)
    const unlock = () => {
      ensureCtx().resume();
      preloader();
    };
    window.addEventListener("pointerdown", unlock, { once: true });
    return () => window.removeEventListener("pointerdown", unlock);
    // eslint-disable-next-line react-hooks/exhaustive-deps
  }, [theme]);

  /* ---------- Synth helpers (used for Pokémon + classic fallback) ---------- */
  const tone = (type = "sine", freq = 440, dur = 0.12, gain = 0.06, tOffset = 0) => {
    const ctx = ensureCtx();
    const t0 = ctx.currentTime + 0.005 + tOffset;
    const osc = ctx.createOscillator();
    const g = ctx.createGain();
    osc.type = type;
    osc.frequency.setValueAtTime(freq, t0);
    g.gain.setValueAtTime(0, t0);
    g.gain.linearRampToValueAtTime(gain, t0 + 0.01);
    g.gain.exponentialRampToValueAtTime(0.0001, t0 + dur);
    osc.connect(g).connect(ctx.destination);
    osc.start(t0);
    osc.stop(t0 + dur + 0.02);
  };

  const slide = (type, start, end, dur = 0.18, gain = 0.055, tOffset = 0) => {
    const ctx = ensureCtx();
    const t0 = ctx.currentTime + 0.005 + tOffset;
    const osc = ctx.createOscillator();
    const g = ctx.createGain();
    osc.type = type;
    osc.frequency.setValueAtTime(start, t0);
    osc.frequency.linearRampToValueAtTime(end, t0 + dur);
    g.gain.setValueAtTime(0, t0);
    g.gain.linearRampToValueAtTime(gain, t0 + 0.01);
    g.gain.exponentialRampToValueAtTime(0.0001, t0 + dur);
    osc.connect(g).connect(ctx.destination);
    osc.start(t0);
    osc.stop(t0 + dur + 0.02);
  };

  // Soft wooden "thock/click" synthesized fallback
  const woodThock = (low = 220, high = 320, dur = 0.06, gain = 0.07) => {
    const ctx = ensureCtx();
    const t0 = ctx.currentTime + 0.005;
    const o1 = ctx.createOscillator();
    const o2 = ctx.createOscillator();
    const g = ctx.createGain();
    o1.type = "triangle"; o2.type = "sine";
    o1.frequency.setValueAtTime(low, t0);
    o2.frequency.setValueAtTime(high, t0);
    g.gain.setValueAtTime(0, t0);
    g.gain.linearRampToValueAtTime(gain, t0 + 0.008);
    g.gain.exponentialRampToValueAtTime(0.0001, t0 + dur);
    o1.connect(g); o2.connect(g); g.connect(ctx.destination);
    o1.start(t0); o2.start(t0);
    o1.stop(t0 + dur + 0.02); o2.stop(t0 + dur + 0.02);
  };

  const playBuffer = (name) => {
    const buf = buffersRef.current[name];
    if (!buf) {
      // synth fallback
      if (name === "capture") woodThock(180, 260, 0.09, 0.09);
      else if (name === "move") woodThock(220, 320, 0.06, 0.07);
      else if (name === "check") { woodThock(340, 480, 0.08, 0.08); woodThock(260, 360, 0.08, 0.06); }
      else if (name === "win") { tone("triangle", 660, 0.12, 0.06); tone("triangle", 784, 0.12, 0.06, 0.12); }
      else if (name === "draw") { tone("sine", 520, 0.1, 0.05); tone("sine", 520, 0.1, 0.05, 0.12); }
      else if (name === "lose") { tone("sine", 380, 0.12, 0.05); tone("sine", 280, 0.12, 0.05, 0.12); }
      return;
    }
    const ctx = ensureCtx();
    const src = ctx.createBufferSource();
    src.buffer = buf;
    const g = ctx.createGain();
    g.gain.value = 0.8;
    src.connect(g).connect(ctx.destination);
    src.start();
  };

  /* ---------- Public API ---------- */
  const play = (pieceType, isCapture = false) => {
    if (theme === "classic") {
      playBuffer(isCapture ? "capture" : "move");
      return;
    }
    // Pokémon mode (retro)
    const base = { p: 400, n: 520, b: 480, r: 360, q: 600, k: 300 }[pieceType] || 440;
    if (isCapture) {
      tone("sawtooth", base * 0.9, 0.09, 0.06);
      tone("square", base * 1.15, 0.11, 0.07, 0.06);
    } else {
      const map = {
        p: () => tone("sine", base, 0.08, 0.05),
        n: () => { tone("triangle", base, 0.06, 0.05); tone("triangle", base * 1.06, 0.05, 0.04, 0.055); },
        b: () => { tone("sine", base, 0.07, 0.045); tone("sine", base * 1.08, 0.07, 0.04, 0.07); },
        r: () => tone("square", base * 0.95, 0.09, 0.055),
        q: () => { tone("sawtooth", base, 0.08, 0.05); tone("sawtooth", base * 1.1, 0.08, 0.045, 0.06); },
        k: () => tone("sine", base * 0.9, 0.12, 0.055),
      };
      (map[pieceType] || map.p)();
    }
  };

  const playSpecial = (kind) => {
    if (theme === "classic") {
      if (kind === "check") playBuffer("check");
      else playBuffer("move");
      return;
    }
    if (kind === "castle-k") { slide("square", 320, 420, 0.18); tone("triangle", 540, 0.09, 0.05, 0.18); }
    if (kind === "castle-q") { slide("square", 420, 320, 0.18); tone("triangle", 500, 0.09, 0.05, 0.18); }
    if (kind === "enpassant") { tone("sine", 420, 0.05, 0.045); tone("triangle", 370, 0.08, 0.05, 0.05); }
    if (kind === "check") { tone("square", 700, 0.06, 0.05); tone("square", 640, 0.06, 0.045, 0.07); }
  };

  const jingle = (result) => {
    if (theme === "classic") {
      if (result === "win") playBuffer("win");
      else if (result === "lose") playBuffer("lose");
      else playBuffer("draw");
      return;
    }
    if (result === "win") {
      tone("triangle", 660, 0.12, 0.06);
      tone("triangle", 784, 0.12, 0.06, 0.12);
      tone("square", 988, 0.14, 0.07, 0.24);
    } else if (result === "lose") {
      tone("square", 520, 0.12, 0.05);
      tone("square", 440, 0.12, 0.05, 0.12);
      tone("sine", 330, 0.14, 0.05, 0.24);
    } else {
      tone("triangle", 520, 0.1, 0.05);
      tone("triangle", 520, 0.1, 0.05, 0.12);
      tone("triangle", 520, 0.1, 0.05, 0.24);
    }
  };

  return { play, playSpecial, jingle, resume: () => ensureCtx().resume() };
}

/* ------------------------- Legend + Move list ------------------------- */
function PieceLegend({ themePokemon }) {
  const names = { k: "King", q: "Queen", r: "Rook", b: "Bishop", n: "Knight", p: "Pawn" };
  return (
    <div className="rounded-2xl p-4 bg-white/70 dark:bg-neutral-900/60 backdrop-blur-xl shadow-xl border border-white/30 dark:border-black/20">
      <div className="text-lg font-semibold mb-3">Piece Lookup</div>
      <div className="grid grid-cols-2 gap-3">
        {["w", "b"].map((color) => (
          <div key={color} className="space-y-2">
            <div className="text-sm opacity-70">{color === "w" ? "White" : "Black"}</div>
            {Object.keys(names).map((t) => (
              <div key={t} className="flex items-center gap-2 rounded-xl p-2 bg-neutral-100/70 dark:bg-neutral-800/50">
                {themePokemon ? (
                  <div
                    className="h-10 w-10 rounded-lg bg-center bg-no-repeat [background-size:cover] shadow"
                    style={{ backgroundImage: `url(${POKEMON_SPRITES[color][t]})` }}
                    aria-label={`${color}-${t}`}
                  />
                ) : (
                  <div className="h-10 w-10 rounded-lg flex items-center justify-center bg-white/70 dark:bg-neutral-700/70 shadow">
                    <span
                      className={`leading-none ${color === "w" ? "text-black" : "text-neutral-800"} dark:text-white`}
                      style={{
                        fontSize: t === "p" ? 24 : 28, // <-- smaller pawn in legend
                        transform: t === "p" ? "translateY(1px)" : undefined,
                        textShadow: color === "w" ? "0 1px 0 #fff, 0 2px 2px rgba(0,0,0,.25)" : "0 1px 0 #000",
                      }}
                    >
                      {CLASSIC_GLYPHS[color][t]}
                    </span>
                  </div>
                )}
                <div className="text-sm">{names[t]}</div>
              </div>
            ))}
          </div>
        ))}
      </div>
    </div>
  );
}

function MoveList({ game }) {
  const history = game.history({ verbose: true });
  const pairs = [];
  for (let i = 0; i < history.length; i += 2) {
    pairs.push([history[i], history[i + 1]]);
  }
  return (
    <div className="rounded-2xl p-4 bg-white/70 dark:bg-neutral-900/60 backdrop-blur-xl shadow-xl border border-white/30 dark:border-black/20">
      <div className="text-lg font-semibold mb-3">Move List</div>
      <ol className="text-sm max-h-[300px] overflow-auto pr-1">
        {pairs.map((p, idx) => (
          <li key={idx} className="flex items-center justify-between py-1 border-b border-black/5 dark:border-white/5 last:border-none">
            <span className="opacity-70 w-6">{idx + 1}.</span>
            <span className="flex-1">{p[0]?.san || ""}</span>
            <span className="flex-1 text-right">{p[1]?.san || ""}</span>
          </li>
        ))}
      </ol>
    </div>
  );
}

/* ------------------------- Clock ------------------------- */
function formatClock(ms) {
  const neg = ms < 0;
  const v = Math.max(0, Math.floor(ms));
  const m = Math.floor(v / 60000);
  const s = Math.floor((v % 60000) / 1000);
  const tenths = Math.floor((v % 1000) / 100);
  return `${neg ? "-" : ""}${String(m).padStart(2, "0")}:${String(s).padStart(2, "0")}${m < 1 ? "." + tenths : ""}`;
}

function ClockPanel({
  turn,
  wTime, bTime,
  baseMinutes, setBaseMinutes,
  incrementSec, setIncrementSec,
  incMs,
  clockRunning, setClockRunning,
  onApplySettings, onResetClock,
  flagged
}) {
  const active = turn;

  const applyPreset = (m, inc) => {
    setBaseMinutes(m);
    setIncrementSec(inc);
    onApplySettings(m, inc);
  };

  return (
    <div className="rounded-2xl p-4 bg-white/80 dark:bg-neutral-900/70 backdrop-blur-xl shadow-xl border border-white/30 dark:border-black/20 space-y-4">
      <div className="text-lg font-semibold">Clock</div>

      {/* Displays */}
      <div className="grid grid-cols-2 gap-3">
        <div className={["rounded-xl p-3 text-center", active === "w" && !flagged ? "ring-2 ring-emerald-500" : "ring-1 ring-black/10"].join(" ")}>
          <div className="text-xs opacity-70 mb-1">White</div>
          <div className="font-mono text-2xl">{formatClock(wTime)}</div>
        </div>
        <div className={["rounded-xl p-3 text-center", active === "b" && !flagged ? "ring-2 ring-rose-500" : "ring-1 ring-black/10"].join(" ")}>
          <div className="text-xs opacity-70 mb-1">Black</div>
          <div className="font-mono text-2xl">{formatClock(bTime)}</div>
        </div>
      </div>

      {/* Controls */}
      <div className="flex flex-wrap gap-2">
        <Button onClick={() => setClockRunning(v => !v)} variant="outline">{clockRunning ? "Pause" : "Start"}</Button>
        <Button onClick={onResetClock} variant="outline">Reset Clock</Button>
      </div>

      {/* Presets */}
      <div className="space-y-2">
        <div className="text-sm opacity-70">Presets</div>
        <div className="flex flex-wrap gap-2">
          <Button variant="outline" onClick={() => applyPreset(10, 0)}>10 + 0</Button>
          <Button variant="outline" onClick={() => applyPreset(5, 3)}>5 + 3</Button>
          <Button variant="outline" onClick={() => applyPreset(3, 2)}>3 + 2</Button>
          <Button variant="outline" onClick={() => applyPreset(1, 0)}>1 + 0</Button>
        </div>
      </div>

      {/* Custom */}
      <div className="space-y-2">
        <div className="text-sm opacity-70">Custom</div>
        <div className="flex items-center gap-2">
          <input
            type="number" min="0" className="w-20 rounded-lg border px-2 py-1 bg-white/80 dark:bg-neutral-800/70"
            value={baseMinutes} onChange={(e) => onApplySettings(Number(e.target.value), incrementSec)}
          />
          <span className="text-sm">minutes</span>
          <input
            type="number" min="0" className="w-20 rounded-lg border px-2 py-1 bg-white/80 dark:bg-neutral-800/70"
            value={incrementSec} onChange={(e) => onApplySettings(baseMinutes, Number(e.target.value))}
          />
          <span className="text-sm">increment (sec)</span>
        </div>
        <div className="text-xs opacity-70">Increment (+{(incMs/1000)|0}s) is added to the player who just moved.</div>
      </div>
    </div>
  );
}

/* ------------------------- Square ------------------------- */
function Square({
  dark,
  children,
  highlight,
  lastMove,
  canMove,
  illegal,
  checkPulse,
  mateBurst,
  flippedCounter,
  themePokemon,
  onDragOver,
  onDrop,
  onClick,
}) {
  const pokeWatermark =
    "bg-[radial-gradient(circle_at_50%_45%,rgba(0,0,0,0.06)_0%,rgba(0,0,0,0.06)_18%,transparent_19%),linear-gradient(to_bottom,rgba(0,0,0,0.08)_0%,rgba(0,0,0,0.08)_49%,rgba(0,0,0,0.18)_50%,rgba(0,0,0,0.08)_51%,rgba(0,0,0,0.08)_100%)]";
  const baseGradPokemon = dark
    ? "bg-[linear-gradient(135deg,rgba(64,0,0,0.4),rgba(30,0,0,0.7))]"
    : "bg-[linear-gradient(135deg,rgba(255,255,255,0.9),rgba(235,235,235,0.9))]";

  // Classic wood-ish palette
  const baseGradClassic = dark
    ? "bg-[linear-gradient(135deg,#8b5a2b,#5c3b18)]"
    : "bg-[linear-gradient(135deg,#f0d9b5,#e5c79a)]";

  return (
    <motion.div
      onDragOver={onDragOver}
      onDrop={onDrop}
      onClick={onClick}
      animate={illegal ? { x: [0, -6, 6, -3, 3, 0] } : { x: 0 }}
      transition={{ duration: 0.25 }}
      className={[
        "relative aspect-square select-none",
        themePokemon ? baseGradPokemon : baseGradClassic,
        themePokemon ? pokeWatermark : "",
        "transition-all duration-200 ease-in-out",
        highlight ? "ring-4 ring-amber-400 scale-105" : "",
        illegal ? "ring-4 ring-red-400 animate-pulse" : "",
        checkPulse ? "ring-4 ring-red-500 animate-pulse" : "",
        mateBurst ? "ring-8 ring-red-600" : "",
        lastMove ? (dark ? "brightness-95" : "brightness-105") : "",
        canMove ? "cursor-pointer" : "",
        "overflow-hidden shadow-inner",
      ].join(" ")}
      style={{ perspective: 900 }}
    >
      {/* glossy sheen (keep subtle in both themes) */}
      <div className="pointer-events-none absolute inset-0 bg-[radial-gradient(circle_at_25%_20%,rgba(255,255,255,0.18),transparent_45%)]" />
      {/* check pulse overlay */}
      {checkPulse && (
        <div className="pointer-events-none absolute inset-0 bg-[radial-gradient(circle_at_50%_50%,rgba(239,68,68,0.22),transparent_60%)]" />
      )}
      {/* checkmate burst overlay */}
      {mateBurst && (
        <div className="pointer-events-none absolute inset-0">
          <div className="absolute inset-0 animate-[ping_0.9s_ease-out_1] bg-red-500/30" />
        </div>
      )}
      {/* Counter-rotate contents when board flips */}
      <div className={flippedCounter ? "w-full h-full rotate-180" : "w-full h-full"}>{children}</div>
    </motion.div>
  );
}

/* ------------------------- Promotion dialog ------------------------- */
function PromotionDialog({ open, onClose, onPick, color, themePokemon }) {
  const options = ["q", "r", "b", "n"];
  return (
    <Dialog open={open}>
      <DialogContent className="sm:max-w-[420px]">
        <DialogHeader>
          <DialogTitle>Choose a promotion</DialogTitle>
        </DialogHeader>
        <div className="grid grid-cols-4 gap-3">
          {options.map((t) => (
            <button
              key={t}
              onClick={() => onPick(t)}
              className="rounded-2xl shadow p-2 bg-neutral-100 hover:bg-neutral-200 dark:bg-neutral-800 dark:hover:bg-neutral-700"
            >
              <div className="aspect-square rounded-xl flex items-center justify-center bg-white/80 dark:bg-neutral-700/60">
                {themePokemon ? (
                  <div
                    className="w-full h-full rounded-xl [background-size:cover] bg-center"
                    style={{ backgroundImage: `url(${POKEMON_SPRITES[color][t]})` }}
                  />
                ) : (
                  <span className="text-4xl leading-none">{CLASSIC_GLYPHS[color][t]}</span>
                )}
              </div>
              <div className="mt-1 text-center uppercase text-xs text-neutral-600 dark:text-neutral-300">{t}</div>
            </button>
          ))}
        </div>
        <div className="flex justify-end mt-4">
          <Button variant="outline" onClick={onClose}>Cancel</Button>
        </div>
      </DialogContent>
    </Dialog>
  );
}

/* ------------------------- Main component ------------------------- */
export default function PokemonChess() {
  const [game] = useState(() => new Chess());
  const [selected, setSelected] = useState(null);
  const [moves, setMoves] = useState([]);
  const [lastMove, setLastMove] = useState(null);
  const [promotion, setPromotion] = useState({ open: false, from: null, to: null, color: "w" });
  const [illegalFlash, setIllegalFlash] = useState(false);
  const [sfxEnabled, setSfxEnabled] = useState(true);
  const [autoFlip, setAutoFlip] = useState(true);
  const [themePokemon, setThemePokemon] = useState(true); // theme toggle

  // Clock
  const [baseMinutes, setBaseMinutes] = useState(10);
  const [incrementSec, setIncrementSec] = useState(0);
  const incMs = incrementSec * 1000;
  const [wTime, setWTime] = useState(baseMinutes * 60 * 1000);
  const [bTime, setBTime] = useState(baseMinutes * 60 * 1000);
  const [clockRunning, setClockRunning] = useState(false);
  const [flagged, setFlagged] = useState(null);

  // SFX (theme-aware)
  const sfx = useChessSfx(themePokemon ? "pokemon" : "classic");
  const dragFrom = useRef(null);

  // stable IDs for animation
  const pieceIds = useRef(new Map());
  const idCounter = useRef(1);

  const assignIdsFromGame = () => {
    pieceIds.current.clear();
    const board = game.board();
    for (let r = 0; r < 8; r++) {
      for (let c = 0; c < 8; c++) {
        const file = String.fromCharCode(97 + c);
        const rank = 8 - r;
        const sq = `${file}${rank}`;
        const piece = board[r][c];
        if (piece) {
          pieceIds.current.set(sq, `${piece.color}-${piece.type}-${idCounter.current++}`);
        }
      }
    }
  };

  const squares = useBoard(game);
  const turn = game.turn();
  const inCheck = game.inCheck();
  const gameOver = game.isGameOver() || !!flagged;

  const flipped = autoFlip && turn === "b";

  // King squares for effects
  const checkedColor = inCheck && !gameOver ? game.turn() : null;
  const checkedKingSq = checkedColor
    ? squares.find((s) => s.piece && s.piece.type === "k" && s.piece.color === checkedColor)?.square
    : null;

  const isMate = !flagged && game.isGameOver() && game.isCheckmate();
  const matedColor = isMate ? game.turn() : null;
  const matedKingSq = isMate
    ? squares.find((s) => s.piece && s.piece.type === "k" && s.piece.color === matedColor)?.square
    : null;

  const legalMovesFrom = (from) => game.moves({ square: from, verbose: true }).map((m) => m.to);

  useEffect(() => {
    assignIdsFromGame();
    const unlock = () => sfx.resume();
    window.addEventListener("pointerdown", unlock, { once: true });
    return () => window.removeEventListener("pointerdown", unlock);
    // eslint-disable-next-line react-hooks/exhaustive-deps
  }, []);

  // clock
  useEffect(() => {
    if (!clockRunning || gameOver) return;
    let prev = performance.now();
    const tick = () => {
      const now = performance.now();
      const delta = now - prev;
      prev = now;
      if (turn === "w") setWTime((t) => Math.max(0, t - delta));
      else setBTime((t) => Math.max(0, t - delta));
      raf = requestAnimationFrame(tick);
    };
    let raf = requestAnimationFrame(tick);
    return () => cancelAnimationFrame(raf);
  }, [clockRunning, turn, gameOver]);

  useEffect(() => {
    if (clockRunning && wTime <= 0 && !flagged) {
      setFlagged("w"); setClockRunning(false); sfx.jingle("lose");
    }
  }, [wTime, clockRunning, flagged, sfx]);
  useEffect(() => {
    if (clockRunning && bTime <= 0 && !flagged) {
      setFlagged("b"); setClockRunning(false); sfx.jingle("lose");
    }
  }, [bTime, clockRunning, flagged, sfx]);

  const applyClockSettings = (m, inc) => {
    const baseMs = Math.max(0, m) * 60 * 1000;
    setBaseMinutes(m);
    setIncrementSec(inc);
    setClockRunning(false);
    setFlagged(null);
    setWTime(baseMs);
    setBTime(baseMs);
  };
  const resetClock = () => applyClockSettings(baseMinutes, incrementSec);

  const tryMove = (from, to, promoPiece) => {
    if (flagged) return false;
    const piece = game.get(from);
    if (!piece) return false;

    // promotion
    const isPawn = piece.type === "p";
    const isWhite = piece.color === "w";
    const targetRank = isWhite ? "8" : "1";
    if (isPawn && to.endsWith(targetRank) && !promoPiece) {
      setPromotion({ open: true, from, to, color: piece.color });
      return false;
    }

    try {
      const move = game.move({ from, to, promotion: promoPiece || "q" });
      if (move) {
        // IDs for animation
        const id = pieceIds.current.get(from);
        pieceIds.current.delete(from);

        if (move.flags?.includes("e")) {
          const file = move.to[0];
          const rank = parseInt(move.to[1], 10) + (piece.color === "w" ? -1 : 1);
          pieceIds.current.delete(`${file}${rank}`);
        } else if (move.captured) {
          pieceIds.current.delete(move.to);
        }

        if (move.flags?.includes("k") || move.flags?.includes("q")) {
          const y = move.to[1];
          if (move.flags.includes("k")) {
            const rid = pieceIds.current.get(`h${y}`);
            if (rid) { pieceIds.current.delete(`h${y}`); pieceIds.current.set(`f${y}`, rid); }
          } else {
            const rid = pieceIds.current.get(`a${y}`);
            if (rid) { pieceIds.current.delete(`a${y}`); pieceIds.current.set(`d${y}`, rid); }
          }
        }

        if (id) pieceIds.current.set(move.to, id);
        else pieceIds.current.set(move.to, `${piece.color}-${piece.type}-${idCounter.current++}`);

        setLastMove({ from: move.from, to: move.to });
        setSelected(null);
        setMoves([]);

        // increment for mover
        if (clockRunning && incMs > 0) {
          if (piece.color === "w") setWTime((t) => t + incMs);
          else setBTime((t) => t + incMs);
        }

        // sounds
        const captured = !!move.captured;
        if (move.flags?.includes("k")) sfx.playSpecial("castle-k");
        else if (move.flags?.includes("q")) sfx.playSpecial("castle-q");
        else if (move.flags?.includes("e")) sfx.playSpecial("enpassant");
        else if (sfxEnabled) sfx.play(piece.type, captured);
        if (game.inCheck() && !game.isGameOver()) sfx.playSpecial("check");
        if (game.isGameOver()) sfx.jingle(game.isCheckmate() ? "win" : "draw");

        return true;
      }
    } catch {
      /* ignore invalid */
    }

    setIllegalFlash(true);
    setTimeout(() => setIllegalFlash(false), 220);
    return false;
  };

  const onSquareClick = (sq) => {
    if (selected === sq) {
      setSelected(null);
      setMoves([]);
      return;
    }
    if (selected) {
      tryMove(selected, sq);
    }
    const p = game.get(sq);
    if (p) {
      const lm = legalMovesFrom(sq);
      setSelected(sq);
      setMoves(lm);
      if (lm.length === 0) {
        setIllegalFlash(true);
        setTimeout(() => setIllegalFlash(false), 220);
      }
    } else {
      setSelected(null);
      setMoves([]);
    }
  };

  const onPickPromotion = (t) => {
    const { from, to } = promotion;
    setPromotion({ open: false, from: null, to: null, color: "w" });
    tryMove(from, to, t);
  };

  const resetBoard = () => {
    game.reset();
    setSelected(null);
    setMoves([]);
    setLastMove(null);
    setPromotion({ open: false, from: null, to: null, color: "w" });
    assignIdsFromGame();
    setFlagged(null);
  };
  const undo = () => {
    game.undo();
    setSelected(null);
    setMoves([]);
    setPromotion({ open: false, from: null, to: null, color: "w" });
    assignIdsFromGame();
  };

  return (
    <div className="min-h-screen w-full bg-[radial-gradient(circle_at_20%_10%,rgba(255,255,255,0.6),transparent_35%),linear-gradient(135deg,#ef4444,#f59e0b_40%,#fde047_100%)] dark:bg-[radial-gradient(circle_at_20%_10%,rgba(255,255,255,0.08),transparent_35%),linear-gradient(135deg,#111827,#1f2937_40%,#111827_100%)] p-4 sm:p-8">
      <div className="mx-auto max-w-6xl">
        <Card className="rounded-3xl shadow-2xl border-0 overflow-hidden bg-white/80 dark:bg-neutral-900/70 backdrop-blur-xl">
          <CardHeader className="flex items-center justify-between sm:flex-row gap-4">
            <CardTitle className="text-2xl sm:text-3xl font-bold tracking-tight flex items-center gap-3">
              <span
                className="inline-block h-8 w-8 rounded-xl shadow"
                style={{ backgroundImage: `url(${themePokemon ? POKEMON_SPRITES[turn].q : POKEMON_SPRITES[turn].p})`, backgroundSize: "cover" }}
              />
              {themePokemon ? "Pokémon Chess" : "Classic Chess"}
            </CardTitle>
            <div className="flex flex-wrap items-center gap-2">
              <Button variant="outline" onClick={undo}>Undo</Button>
              <Button onClick={resetBoard}>New Game</Button>
              <Button variant="outline" onClick={() => { setSfxEnabled((v) => !v); sfx.resume(); }}>
                {sfxEnabled ? "Mute" : "Unmute"}
              </Button>
              <Button variant="outline" onClick={() => setAutoFlip(v => !v)}>
                {autoFlip ? "Auto Flip: On" : "Auto Flip: Off"}
              </Button>
              <Button variant="outline" onClick={() => setThemePokemon(v => !v)}>
                {themePokemon ? "Theme: Pokémon" : "Theme: Classic"}
              </Button>
            </div>
          </CardHeader>

          <CardContent>
            <div className="grid lg:grid-cols-[1fr_320px] gap-8 items-start">
              {/* Board */}
              <div className="w-full">
                <div className="relative mx-auto w-full max-w-[min(90vw,700px)]">
                  {/* Outer bezel changes with theme */}
                  <motion.div
                    initial={{ opacity: 0, y: 10 }}
                    animate={{ opacity: 1, y: 0 }}
                    transition={{ duration: 0.4 }}
                    className={[
                      "rounded-[28px] p-3 shadow-[0_20px_40px_rgba(0,0,0,0.35)]",
                      themePokemon
                        ? "bg-[radial-gradient(circle_at_10%_10%,rgba(255,255,255,0.7),transparent_30%),linear-gradient(135deg,rgba(220,38,38,0.9),rgba(185,28,28,0.9))] ring-4 ring-black/20"
                        : "bg-[linear-gradient(135deg,#3b2c22,#1f140e)] ring-4 ring-amber-900/40",
                    ].join(" ")}
                  >
                    {/* Board grid (rotates on flip) */}
                    <div
                      className={[
                        "relative grid grid-cols-8 gap-0 rounded-[22px] overflow-hidden",
                        themePokemon
                          ? "ring-8 ring-white/80 dark:ring-black/60 shadow-[inset_0_2px_8px_rgba(255,255,255,0.25),0_20px_40px_rgba(0,0,0,0.35)]"
                          : "ring-[10px] ring-amber-700/40 shadow-[inset_0_2px_10px_rgba(0,0,0,0.35),0_20px_40px_rgba(0,0,0,0.35)]",
                        "transition-transform duration-300",
                        autoFlip && turn === "b" ? "rotate-180" : "",
                      ].join(" ")}
                      style={{ perspective: 1000 }}
                    >
                      <LayoutGroup id="board">
                        {useBoard(game).map(({ square, piece, row, col }) => {
                          const dark = (row + col) % 2 === 1;
                          const isSelected = selected === square;
                          const canMove = moves.includes(square);
                          const isLast = lastMove && (lastMove.from === square || lastMove.to === square);
                          const pid = piece ? (pieceIds.current.get(square) || `${piece.color}-${piece.type}-temp-${square}`) : null;
                          const isMovingDest = lastMove?.to === square;

                          return (
                            <Square
                              key={square}
                              dark={dark}
                              highlight={isSelected}
                              illegal={illegalFlash && isSelected}
                              checkPulse={checkedKingSq === square}
                              mateBurst={matedKingSq === square}
                              lastMove={isLast}
                              canMove={canMove}
                              flippedCounter={autoFlip && turn === "b"}
                              themePokemon={themePokemon}
                              onClick={() => onSquareClick(square)}
                              onDragOver={(e) => e.preventDefault()}
                              onDrop={(e) => {
                                e.preventDefault();
                                const from = dragFrom.current || e.dataTransfer.getData("text/plain");
                                if (!from) return;
                                tryMove(from, square);
                                dragFrom.current = null;
                              }}
                            >
                              {canMove && !piece && (
                                <div className="absolute inset-0 flex items-center justify-center">
                                  <div className={`w-4 h-4 rounded-full ${themePokemon ? "bg-amber-500/70" : "bg-emerald-500/70"}`} />
                                </div>
                              )}
                              {canMove && piece && (
                                <div className={`absolute inset-0 pointer-events-none ring-4 ${themePokemon ? "ring-amber-500/70" : "ring-emerald-500/70"}`} />
                              )}

                              {piece && (
                                <motion.div
                                  layout
                                  layoutId={pid}
                                  transition={{ type: "spring", stiffness: 500, damping: 34, mass: 0.6 }}
                                  animate={isMovingDest ? { z: [0, 18, 0], rotateX: [0, 8, 0], rotateY: [0, -6, 0] } : { z: 0, rotateX: 0, rotateY: 0 }}
                                  style={{ transformStyle: "preserve-3d" }}
                                  className={[
                                    "relative w-full h-full",
                                    "transition-transform hover:scale-[1.03]",
                                    isSelected ? "scale-105" : "",
                                    themePokemon ? "[background-size:cover] bg-center bg-no-repeat drop-shadow-[0_4px_8px_rgba(0,0,0,0.35)]" : "flex items-center justify-center",
                                  ].join(" ")}
                                  draggable={piece.color === turn}
                                  onDragStart={(e) => {
                                    dragFrom.current = square;
                                    e.dataTransfer.setData("text/plain", square);
                                    const crt = document.createElement("div");
                                    crt.style.width = "1px";
                                    crt.style.height = "1px";
                                    document.body.appendChild(crt);
                                    e.dataTransfer.setDragImage(crt, 0, 0);
                                    setTimeout(() => document.body.removeChild(crt), 0);
                                  }}
                                  {...(themePokemon
                                    ? { style: { backgroundImage: `url(${POKEMON_SPRITES[piece.color][piece.type]})`, transformStyle: "preserve-3d" } }
                                    : {})}
                                >
                                  {!themePokemon && (
                                    <span
                                      className={`${piece.color === "w" ? "text-white" : "text-black"} leading-none select-none`}
                                      style={{
                                        /* Base size, with a smaller scale for pawns */
                                        fontSize: `clamp(${piece.type === "p" ? "32px" : "36px"}, 10vw, ${piece.type === "p" ? "78px" : "88px"})`,
                                        transform: piece.type === "p" ? "translateY(2px)" : undefined, // nudge down slightly
                                        textShadow:
                                          piece.color === "w"
                                            ? "0 1px 1px rgba(0,0,0,.35), 0 0 10px rgba(0,0,0,.2)"
                                            : "0 1px 0 rgba(255,255,255,.3)",
                                      }}
                                    >
                                      {CLASSIC_GLYPHS[piece.color][piece.type]}
                                    </span>
                                  )}
                                </motion.div>
                              )}
                            </Square>
                          );
                        })}
                      </LayoutGroup>
                    </div>
                  </motion.div>
                </div>

                {/* Status row */}
                <div className="mt-4 flex items-center justify-between text-sm text-neutral-700 dark:text-neutral-300">
                  <div className="flex items-center gap-3">
                    {themePokemon ? (
                      <span className="relative h-3 w-40 rounded-full bg-neutral-200 dark:bg-neutral-800 overflow-hidden">
                        <span className={["absolute inset-y-0 transition-all", turn === "w" ? "left-0 w-1/2 bg-green-500" : "right-0 w-1/2 bg-red-500"].join(" ")} />
                      </span>
                    ) : null}
                    <span className="inline-flex items-center gap-2 px-2 py-1 rounded-full bg-neutral-200/60 dark:bg-neutral-800/60">
                      {themePokemon && (
                        <span
                          className="inline-block h-4 w-4 rounded shadow"
                          style={{ backgroundImage: `url(${POKEMON_SPRITES[turn].p})`, backgroundSize: "cover" }}
                        />
                      )}
                      {gameOver
                        ? flagged
                          ? `${flagged === "w" ? "White" : "Black"} flagged`
                          : (game.isCheckmate() ? "Checkmate" : "Draw")
                        : `${turn === "w" ? "White" : "Black"} to move`}
                    </span>
                    {inCheck && !gameOver && <span>• In check!</span>}
                  </div>
                  <div className="opacity-70">{themePokemon ? "Made with chess.js + PokeAPI sprites" : "Made with chess.js"}</div>
                </div>
              </div>

              {/* Right column: clock + legend + moves */}
              <div className="space-y-4">
                <ClockPanel
                  turn={turn}
                  wTime={wTime} bTime={bTime}
                  baseMinutes={baseMinutes} setBaseMinutes={setBaseMinutes}
                  incrementSec={incrementSec} setIncrementSec={setIncrementSec}
                  incMs={incMs}
                  clockRunning={clockRunning} setClockRunning={setClockRunning}
                  onApplySettings={applyClockSettings}
                  onResetClock={resetClock}
                  flagged={flagged}
                />
                <PieceLegend themePokemon={themePokemon} />
                <MoveList game={game} />
              </div>
            </div>
          </CardContent>
        </Card>
      </div>

      <PromotionDialog
        open={promotion.open}
        color={promotion.color}
        themePokemon={themePokemon}
        onClose={() => setPromotion({ open: false, from: null, to: null, color: "w" })}
        onPick={onPickPromotion}
      />
    </div>
  );
}
